## Import statements

In [ ]:
from ds_util.config import cfg, get_project_root
from ds_util.data_accessors import snowflake_dataframe_from_sql, snowflake_execute_query

#%matplotlib inline 
#import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
!jupyter nbextension enable --py widgetsnbextension

## Data Ingestion

In [ ]:
df = snowflake_dataframe_from_sql("""
select store_order_id,
order_id,
store_id,
store_city,
store_state 
from prod.reporting.store_orders_completed_core 
limit 1000""")

## Data Processing

In [ ]:
common_timestamp_columns = ['date', 'created_at','updated_at','dt','timestamp','event_time']
for column in df.columns:
    if str(column).lower() in common_timestamp_columns:
        print(f"Updating {column} from object to datetime")
        df[column] = pd.to_datetime(df[column])

## EDA

Basic column information and counts

In [ ]:
df.info()

##### Data profiling
This can be written to disk using the to_file(filename) command or it can be displayed directly

In [ ]:
pr = ProfileReport(df, explorative = True)
pr.to_file("00-EDA-template-profile.html")

#### Pairwise analysis

In [ ]:
# Set the default theme for seaborn
sns.set_theme()
sns.pairplot(df[df.columns])

# Generate Great Expectations

#### Import the necessary packages

In [ ]:
from datetime import datetime
import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.data_context.types.resource_identifiers import ValidationResultIdentifier
from great_expectations.dataset import (
    PandasDataset,
    MetaPandasDataset,
)
from great_expectations.profile import BasicSuiteBuilderProfiler

#### Convert your dataframe into a GE dataframe

In [ ]:
ge_df = ge.from_pandas(df)

#### Load the template 
The directory here is built by the command `great_expectations init,` but the cookiecutter template should have this already populated for you. This is where your expectations will get saved, and this is where Dagster will pull them from to run validation against flows.

In [ ]:
context = ge.data_context.DataContext(context_root_dir=get_project_root().joinpath("dagster/great_expectations"))
# Name your suite whatever you want. This is the project template name
expectation_suite_name = "{{cookiecutter.project_slug}}.basic.warning"
# The arg overwrite_existing=True can be added if you want to overwrite stuff that is already there
es = context.create_expectation_suite(expectation_suite_name)

#### Configure the data you will use for creating expectations

In [ ]:
# datasource can be changed to be non-pandas (a local file, a file in s3, another database)
# and the dataset will need to change accordingly
batch_kwargs = {
    'dataset': df,
    'datasource': "pandas"
}
batch = context.get_batch(batch_kwargs, expectation_suite_name)
#Test that the batch has data and see what it is
batch.head()

### Start to explore expectations

In [ ]:
included_columns = [
    "store_order_id",
    "order_id",
    "store_id",
    "store_city",
    "store_state",
]
scaffold_config = {
    "included_columns" : included_columns
}
# This generates a bunch of expectations against your data that generally are bad, 
# but they give you a format to work against
suite, evr = BasicSuiteBuilderProfiler().profile(batch, profiler_configuration=scaffold_config)

### A few sample explorations to get started

In [ ]:
batch.expect_column_quantile_values_to_be_between(
    column="store_order_id",
    allow_relative_error=False,
    quantile_ranges={
        "quantiles": [0.05, 0.25, 0.5, 0.75, 0.95],
        "value_ranges": [
            [39.0, 41.0],
            [127.0, 129.0],
            [391.0, 393.0],
            [1245.0, 1247.0],
            [5303.0, 5305.0],
        ],
    },
)

In [ ]:
batch.expect_column_values_to_not_be_null(column="order_id", mostly=0.898)

In [ ]:
batch.expect_column_values_to_not_be_in_set(column="store_state", value_set=["fakeville"] )

In [ ]:
# Persist the expectations
context.save_expectation_suite(suite, expectation_suite_name)

### Test expectations by running validate against a dataset

In [ ]:
# validate
results = context.run_validation_operator("action_list_operator", assets_to_validate=[batch])

# build the documentation for the result
validation_result_identifier = results.list_validation_result_identifiers()[0]
context.build_data_docs()
data_docs_urls = context.get_docs_sites_urls(
            resource_identifier=validation_result_identifier,
            only_if_exists=False,
        )
urls_to_open = [site["site_url"] for site in data_docs_urls]
url = urls_to_open[0].replace('file:///work/', 'http://localhost:8888/view/')

# You'll need to paste this in your browser if you are running this in the container
# if needed you can add `+f'?token={access_token}' to the end of the above line where the access token 
# is generated by your notebook server 
url